In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
folder = '../../outputs/ihme_seir/compartmental_paper/20200905_191750_tp'

regions = ['Delhi', 'New York City', 'Italy', 'Pune', 'Los Angeles']

In [ ]:
df_list = []
for region in regions:
    shift_list = []
    for s in range(0,22,2):
        tp_list = []
        for tp in range(4,42,2):
            runs_list = []
            for run in range(0,10):
                run_df = pd.read_csv(f'{folder}/{region}/{s}/train_{tp}/run_{run}/sir_loss.csv', index_col=0) 
                run_df.reset_index(inplace=True)
                run_df.rename(columns={'index':'split'}, inplace=True)
                run_df.insert(0, 'run', run+1)
                runs_list.append(run_df)
            runs_df = pd.concat(runs_list, axis=0).reset_index(drop=True)
            runs_df.insert(0, 'train_period', tp)
            tp_list.append(runs_df)
        tp_df = pd.concat(tp_list, axis=0).reset_index(drop=True)
        tp_df.insert(0, 'shift', s)
        shift_list.append(tp_df)
    shift_df = pd.concat(shift_list, axis=0).reset_index(drop=True)
    shift_df.insert(0, 'region', region)
    df_list.append(shift_df)
df = pd.concat(df_list, axis=0).reset_index(drop=True)

In [ ]:
df

In [ ]:
df = df.rename(columns={'total_infected': 'mape'})
df.to_csv('train_period.csv', index=False)

In [ ]:
df1 = df[df['split'] == 'val']
df1 = df1[df1['train_period'] > 8]

In [ ]:
%matplotlib inline
sns.set(font_scale=1.5)
fig, ax = plt.subplots(figsize=(20,20))
sns.pointplot(x='train_period', y='mape', hue='region', data=df1, dodge=0.1, 
              ax=ax)